# **Custom Model**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image

In [2]:
# Custom CNN Model with More Layers and Batch Normalization
class nULLCNN(nn.Module):
    def __init__(self, num_classes):
        super(nULLCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * 14 * 14, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = x.view(-1, 256 * 14 * 14)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [3]:
# Paths to the dataset
train_path = '/kaggle/input/sugarcane-leaf-disease/Sugarcane Leaf Disease/Train'
test_path = '/kaggle/input/sugarcane-leaf-disease/Sugarcane Leaf Disease/Test'

In [4]:
# Data transformations
transformations = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
# Load the datasets
train_dataset = datasets.ImageFolder(root=train_path, transform=transformations)
test_dataset = datasets.ImageFolder(root=test_path, transform=transformations)

In [6]:
# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [7]:
# Instantiate the model
num_classes = len(train_dataset.classes)
model = nULLCNN(num_classes)

In [8]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [9]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

nULLCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=5, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [10]:
# Training the model
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

Epoch 1/15 - Loss: 1.2372, Accuracy: 52.30%
Epoch 2/15 - Loss: 0.7973, Accuracy: 69.34%
Epoch 3/15 - Loss: 0.6059, Accuracy: 78.21%
Epoch 4/15 - Loss: 0.5955, Accuracy: 76.82%
Epoch 5/15 - Loss: 0.4576, Accuracy: 83.70%
Epoch 6/15 - Loss: 0.4178, Accuracy: 84.74%
Epoch 7/15 - Loss: 0.4802, Accuracy: 83.21%
Epoch 8/15 - Loss: 0.3202, Accuracy: 88.61%
Epoch 9/15 - Loss: 0.2769, Accuracy: 90.44%
Epoch 10/15 - Loss: 0.2502, Accuracy: 90.89%
Epoch 11/15 - Loss: 0.2500, Accuracy: 91.48%
Epoch 12/15 - Loss: 0.2855, Accuracy: 89.80%
Epoch 13/15 - Loss: 0.2256, Accuracy: 93.07%
Epoch 14/15 - Loss: 0.3910, Accuracy: 88.31%
Epoch 15/15 - Loss: 0.3654, Accuracy: 88.16%


In [11]:
# Testing the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 85.26%


In [12]:
torch.save(model.state_dict(), 'custom_model.pth')

# **Predict Class for Custom Data(Pytorch)**

In [13]:
model = nULLCNN(num_classes)
model.load_state_dict(torch.load('/kaggle/working/custom_model.pth', map_location=device))
model.to(device)
model.eval()

nULLCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=5, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [14]:
# Function to predict the class of a single image
def predict_image(image_path, model, transformations, class_names):
    image = Image.open(image_path).convert('RGB')
    image = transformations(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)

    return class_names[predicted.item()]

In [15]:
class_names = train_dataset.classes
# Predict the class for a custom image
image_path = '/kaggle/input/sugarcane-leaf-disease/Sugarcane Leaf Disease/Train/Mosaic/mosaic (10).jpeg'
predicted_class = predict_image(image_path, model, transformations, class_names)
print(f"Predicted class: {predicted_class}")

Predicted class: Mosaic


In [16]:
# !pip install onnx
# !pip install onnx-tf
# !pip install tensorflow

# **Load Pytorch Model**

In [17]:
model.load_state_dict(torch.load('/kaggle/working/custom_model.pth'))
model.eval()

nULLCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=5, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [18]:
# Dummy input for the model on the same device
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# **Export pytorch to ONNX**

In [19]:
# Export the model to ONNX
torch.onnx.export(model, dummy_input, '/kaggle/working/custom_model.onnx', input_names=['input'], output_names=['output'], opset_version=11)

# **Export Tensorflow Model**

In [20]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model = onnx.load('/kaggle/working/custom_model.onnx')

# Convert the ONNX model to TensorFlow
tf_rep = prepare(onnx_model)

# Export the TensorFlow model
tf_rep.export_graph('/kaggle/working/custom_model_tensorflow')

2024-05-18 09:36:15.872468: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 09:36:15.872560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 09:36:15.998797: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our Tens

# **Predict Class for Custom Data(Tensorflow)**

In [21]:
import tensorflow as tf
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

In [22]:
# Load the TensorFlow model
model = tf.saved_model.load('/kaggle/working/custom_model_tensorflow')

In [23]:
# Define your class indices
class_indices = {0: 'Healthy', 1: 'Mosaic', 2: 'RedRot', 3: 'Rust', 4: 'Yellow'}

In [24]:
# Function to preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path)
    transformations = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transformations(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image.numpy()

In [25]:
# Path to the custom image
image_path = '/kaggle/input/sugarcane-leaf-disease/Sugarcane Leaf Disease/Train/Mosaic/mosaic (10).jpeg'
# Preprocess the image
input_tensor = preprocess_image(image_path)

In [26]:
# Run inference
input_tensor = tf.convert_to_tensor(input_tensor, dtype=tf.float32)
output = model.signatures['serving_default'](input_tensor)

In [27]:
# Get the predicted class index
predicted_class_index = np.argmax(output['output'].numpy(), axis=1)[0]
# Get the class name
predicted_class_name = class_indices[predicted_class_index]

In [28]:
# print(f"Predicted class index: {predicted_class_index}")
print(f"Predicted class name: {predicted_class_name}")

Predicted class name: Mosaic
